In [1]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl import function as fn
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh
import networkx as nx
import matplotlib.pyplot as plt
import random as rand
import numpy as np

#set gpu is available
if th.cuda.is_available():
    device = th.device("cuda")
    print("GPU is available")
else:
    device = th.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [2]:
#operation for neigbors
class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        if self.activation is not None:
            h = self.activation(h)
        return {'h' : h}
    
#gcn layer in network
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        
        g.ndata['h'] = feature
        g.pull(g.nodes())
        g.apply_nodes(self.apply_mod)
        
        return g.ndata.pop('h')
    
#network
class GCN_LL(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCN_LL, self).__init__()
        self.gcn1 = GCN(in_feats, 800, F.relu)
        self.gcn2 = GCN(800, 100, F.relu)
        self.gcn3 = GCN(100, out_feats, F.tanh)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        x = self.gcn2(g, x)
        
        return self.gcn3(g, x)


In [10]:
#loss function

"""Here is an implementation for your similarity_matrix using only matrix operations. It can run on the GPU and is going to be significantly faster than your previous implementation.
https://discuss.pytorch.org/t/build-your-own-loss-function-in-pytorch/235/6
"""

def my_loss(output, labels):
    """
    if nodes with the same label: x^2
    if nodes with different label: 1/x
    """
    
    def similarity_matrix(mat):
        # get the product x * y
        # here, y = x.t()
        r = th.mm(mat, mat.t())
        # get the diagonal elements
        diag = r.diag().unsqueeze(0)
        diag = diag.expand_as(r)
        # compute the distance matrix
        D = diag + diag.t() - 2*r
        return D.sqrt()

    def same_label(y):
        s = y.size(0)
        y_expand = y.unsqueeze(0).expand(s, s)
        Y = y_expand.eq(y_expand.t())
        return Y
    
    sim = similarity_matrix(output)
    same_l = same_label(labels)
    same_l_inv = same_l*(-1) + 1
    
    loss = (th.sum(th.mul(sim, same_l))**2) + (1/th.sum(th.mul(sim, same_l_inv))) 
    
    return loss


In [4]:
#load dataset
data = citegrh.load_cora()
ds_features = th.FloatTensor(data.features) #convert to pytorch data type #######
ds_labels = th.LongTensor(data.labels)
ds_train = th.ByteTensor(data.train_mask)
ds_test = th.ByteTensor(data.test_mask)
ds_g = data.graph

# add self loop for the sum of festures
ds_g.remove_edges_from(nx.selfloop_edges(ds_g))
ds_g = DGLGraph(ds_g)
ds_g.add_edges(ds_g.nodes(), ds_g.nodes())
ds_g.ndata['label'] = ds_labels #used to filter and train, not needed for prediction

m_func = fn.copy_src(src='h', out='m')
m_reduce_func = fn.sum(msg='m', out='h')

ds_g.register_message_func(m_func)
ds_g.register_reduce_func(m_reduce_func)

In [11]:
########### Create Model ############

#constant parameters
DIST_VEC_SIZE = 10

model = GCN_LL(ds_features.size()[1], DIST_VEC_SIZE)


In [12]:
#training
opt = th.optim.Adam(model.parameters(), lr=1e-3)

model.train()
for epoch in range(200):
    r = list(range(int(len(ds_g)*.1))) #80 percent of all the nodes
    rand.shuffle(r)
    
    error = [0]
    acc = [0] #accuracy
    for count in r:
        sub_graph = ds_g.subgraph(ds_g.nodes()[:count])
        sub_graph.register_message_func(m_func)
        sub_graph.register_reduce_func(m_reduce_func)
        
        feats = ds_features[:count]
        labs = ds_labels[:count]
        
        out = model(sub_graph, feats)
        
        loss = my_loss(out, labs)
        error.append(loss)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    print("Epoch {epoch}: \n\tavg error = {np.mean(error)} \n\tlast error = {error[-1]} \n\tavg accuracy = {np.mean(acc)} \n\tlast accuracy = {acc[-1]} ")
    
    if(epoch%44 == 0 and DIST_VEC_SIZE == 2):
        pass

RuntimeError: invalid argument 10: ldb should be at least max(1, 0), but have 0 at C:\w\1\s\tmp_conda_3.7_104508\conda\conda-bld\pytorch_1572950778684\work\aten\src\TH/generic/THBlas.cpp:365

In [89]:
#ds_g.nodes[ds_g.nodes()[-3:]].data['label2'] = ds_labels[-3:]
ds_g.nodes[ds_g.nodes()[-5:]].data

{'label': tensor([1, 1, 1, 0, 2]), 'label2': tensor([0, 0, 1, 0, 2])}